In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import pprint 
import geocoder
import zillow
from xml.etree import ElementTree
%matplotlib inline

zillow_api='your key here'
google_api_key = 'your key here'
fbi_api_key =  '?api_key='+'your key here'

# @TODO add rest of loan info
#https://github.com/fbi-cde/crime-data-frontend crime api
# @TODO need to check the actually tax numbers
# @TODO selector to add to DF to then export as CVS to keep logs on different houses, maybe if address != exist add to df

In [56]:
res = requests.get(f'http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id={zillow_api}&state=wa&city=seattle&childtype=neighborhood')
#http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=<ZWSID>&zpid=48749425

In [67]:


tree = ElementTree.fromstring(res.content)
print(f'http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id={zillow_api}&state=wa&city=seattle&childtype=neighborhood')

http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz17yjac2ov0r_1r7kw&state=wa&city=seattle&childtype=neighborhood


In [75]:
for child in tree.iter('response'):
    print(tree)
tree

<Element '{http://www.zillow.com/static/xsd/RegionChildren.xsd}regionchildren' at 0x11d937688>


<Element '{http://www.zillow.com/static/xsd/RegionChildren.xsd}regionchildren' at 0x11d937688>

## FBI Crime Pulls

In [3]:
fbi_base_url = 'https://api.usa.gov/crime/fbi/sapi/'
prop_crime_url = f'api/summarized/state/{state_abr}/property-crime/2010/2017'
agencies_url = 'api/agencies'

In [4]:
prop_crime_res = requests.get(fbi_base_url + prop_crime_url + fbi_api_key).json()
agencies_res = requests.get(fbi_base_url + agencies_url + fbi_api_key).json()

In [5]:
geo = geocoder.google(f'{city} , {state_abr}',key =google_api_key)
geo_county = geo.county.split(' ', 1)[0].upper()

In [6]:
property_crime_df = pd.DataFrame.from_dict(prop_crime_res['results'])
pro_df = property_crime_df.groupby('ori')
angencies_df = pd.DataFrame.from_dict(agencies_res[f'{state_abr}'])
ang_df_t = angencies_df.T
filtered_df = ang_df_t[ang_df_t['county_name'] == geo_county]

In [16]:
filtered_df

,agency_name,agency_type_name,county_name,division_name,latitude,longitude,nibrs,nibrs_start_date,ori,region_desc,region_name,state_abbr,state_name
AZ0073300,Wickenburg Police Department,City,MARICOPA,Mountain,33.97,-112.732,False,None,AZ0073300,Region IV,West,AZ,Arizona
AZ0070300,Buckeye Police Department,City,MARICOPA,Mountain,33.3753,-112.573,False,None,AZ0070300,Region IV,West,AZ,Arizona
AZ0070700,El Mirage Police Department,City,MARICOPA,Mountain,33.3465,-112.496,False,None,AZ0070700,Region IV,West,AZ,Arizona
AZ0071500,Goodyear Police Department,City,MARICOPA,Mountain,33.4907,-112.362,False,None,AZ0071500,Region IV,West,AZ,Arizona
AZ0071900,Paradise Valley Police Department,City,MARICOPA,Mountain,33.5311,-111.943,False,None,AZ0071900,Region IV,West,AZ,Arizona
AZ0079700,Arizona State University: Main Campus,University or College,MARICOPA,Mountain,33.3465,-112.496,False,None,AZ0079700,Region IV,West,AZ,Arizona
AZ0072900,Tempe Police Department,City,MARICOPA,Mountain,33.3465,-112.496,False,None,AZ0072900,Region IV,West,AZ,Arizona
AZ0072500,Scottsdale Police Department,City,MARICOPA,Mountain,33.5695,-111.885,False,None,AZ0072500,Region IV,West,AZ,Arizona
AZ0072100,Peoria Police Department,City,MARICOPA,Mountain,33.3465,-112.496,False,None,AZ0072100,Region IV,West,AZ,Arizona
AZ0073100,Tolleson Police Department,City,MARICOPA,Mountain,33.4503,-112.265,False,None,AZ0073100,Region IV,West,AZ,Arizona


In [10]:
pro_df.head(50)

,actual,cleared,data_year,offense,ori,state_abbr
0,1245,136,2010,property-crime,AZ0070300,AZ
1,1341,169,2011,property-crime,AZ0070300,AZ
2,1372,214,2012,property-crime,AZ0070300,AZ
3,1328,198,2013,property-crime,AZ0070300,AZ
4,1141,321,2014,property-crime,AZ0070300,AZ
5,1103,259,2015,property-crime,AZ0070300,AZ
6,1025,252,2016,property-crime,AZ0070300,AZ
7,1152,222,2017,property-crime,AZ0070300,AZ
8,7516,1342,2010,property-crime,AZ0070500,AZ
9,7416,1254,2011,property-crime,AZ0070500,AZ


## Unit Info

In [2]:
street = '9107 N 13TH ST'
city = 'phonix' 
state_abr = 'AZ'
zip_code = '85020'
unit_count = 3
avg_mo_rent_per_unit = 833
# @TODO find an api or something to pull zip code based vacancy rates
# vacancy_rate as decimal
vacancy_credit_loss = .15
# Laundry vending parking services etc
other_mo_income = 0

## Monthly Operating Expenses

In [26]:
property_mang_fees = 0
repairs_and_main = 800/12
HOA_fees = 0
replacement_reserve = 50
pest_control = 10

# If you're cool enough or not covered by the property manager
advertising = 0
legal = 0 
accounting = 0

# if covered by the land lord
water = 0
gas_and_electric = 0
garbage = 0
ISP_tele = 0

## Capitalization Rate and Valuation

In [27]:
desired_cap_rate = .08
actual_pur_price = 287200

## Loan

In [28]:
down_payment = actual_pur_price * 0.15
mortage_length = 30
annual_interest_rate = .042


In [29]:
total_rental_income = unit_count * avg_mo_rent_per_unit
vac_loss = (vacancy_credit_loss -1) *-1

real_estate_tax = (.015 * actual_pur_price)/12
rental_insur = (.04 * actual_pur_price)/12

mo_op_arr = np.array([property_mang_fees,
                      repairs_and_main,
                      HOA_fees,
                      replacement_reserve,
                      pest_control,advertising,
                      legal,accounting,water,
                      gas_and_electric,
                      garbage,
                      ISP_tele,
                      real_estate_tax,
                      rental_insur
                      ])

# monthly calc
gross_mo_income = total_rental_income * vac_loss
monthly_operating_expenses = mo_op_arr.sum()

# yearly calc
annual_income = gross_mo_income*12
annual_op_expenses = monthly_operating_expenses*12
annual_net_operating_income = annual_income - annual_op_expenses

loan_amount = actual_pur_price - down_payment
acquisition_cost = actual_pur_price * 0.0175
init_investment = down_payment + acquisition_cost

#funny math team
annual_int = 0 #@TODO CUMIPMT EXCEL func turn to code later
anual_principal = 0 # CUMPRINC(C46/12,C45*12,C43,1,12,0))
monthly_mortgage_payment = np.pmt((annual_interest_rate/12),(mortage_length*12),loan_amount) * -1


yearly_mort_payment = monthly_mortgage_payment * 12

property_valueation  = annual_net_operating_income / desired_cap_rate
actual_cap_rate  =   desired_cap_rate / actual_pur_price

final_mo_expenses = gross_mo_income - monthly_operating_expenses - monthly_mortgage_payment
final_yr_expenses = final_mo_expenses * 12
ROI = final_yr_expenses/init_investment

In [30]:
print(f'ROI: {ROI}, Cap Rate: {actual_cap_rate}, Prop Value: {property_valueation}')

ROI: -0.1278772854453374, Cap Rate: 2.785515320334262e-07, Prop Value: 102172.50000000003


In [31]:
down_payment

43080.0

14325.464693633403